In [4]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import re
import numpy as np
from scipy.signal import find_peaks

In [5]:
# Function: load_data
#
# Can load files with the following filename structures:
#   - size + section/position + other -> e.g. 100_s4_b2_graphtype
#   - placement + size + section/position + other -> e.g. tr_100_s4_b2_graphtype
# Inputs: 
# - data_folder: folder where files can be found
# - chip_name, size, graph_type to load
# Output:
# - data_list, capa_list: data and name of the capacitors that were loaded -> all measured capacitors of a specific size on one chip



# Function to load data: returns a list of all measurements of a specific mmnt type, for a specific chip and capa size
def load_data(data_folder, chip_name, size, graph_type):
    # Liste des dossiers dans le dossier data_folder
    date_folders = os.listdir(data_folder)
    
    data_list = []
    capa_list = []

    # Parcours des dossiers de date
    for date_folder in date_folders:
        # Chemin complet du dossier de date
        date_folder_path = os.path.join(data_folder, date_folder)
        
        # Liste des dossiers de chips dans le dossier de date
        chip_folders = [folder for folder in os.listdir(date_folder_path) if os.path.isdir(os.path.join(date_folder_path, folder))]
        
        # Parcours des dossiers de chips
        for chip_folder in chip_folders:
            # Vérification si le nom de la chip correspond
            if chip_name in chip_folder:
                # Chemin complet du dossier de la chip
                chip_folder_path = os.path.join(date_folder_path, chip_folder)
                
                # Liste des fichiers xls dans le dossier de la chip
                xls_files = [file for file in os.listdir(chip_folder_path) if file.endswith('.xls')]
                
                # Parcours des fichiers xls
                for xls_file in xls_files:
                    # Séparation du nom du fichier
                    file_parts = xls_file.split('_')
                    
                    # if the file name starts with the capa size (eg. 500_s4_4d)
                    if file_parts[0] == size: 
                        file_placement, file_size, file_section, file_pos  = None, file_parts[0], file_parts[1], file_parts[2]
                        capa = '_'.join(map(str,[file_size,file_section,file_pos]))
                    # if the file name starts with the capa placement (e.g. tr_500_s4_4d)
                    elif file_parts[1] == size: 
                        file_placement, file_size, file_section, file_pos = file_parts[0], file_parts[1], file_parts[2], file_parts[3]
                        capa = '_'.join(map(str,[file_placement,file_size,file_section,file_pos]))
                    # unkown file name structure
                    else:
                        capa = None

                 
                    file_graph_type = extract_pattern_in_filename(xls_file, graph_type)
                    if file_graph_type == graph_type and capa is not None:
                        # Chargement des données xls dans une DataFrame
                        xls_path = os.path.join(chip_folder_path, xls_file)
                        data = pd.read_excel(xls_path)
                        
                        capa_list.append(capa)
                        data_list.append(data)
                    
                    file_size = None   
                        
    return data_list, capa_list


# Fonction pour extraire le nom de l'expérience à partir du nom de fichier
def extract_pattern_in_filename(file_name, pattern):
    # Expression régulière pour rechercher le motif "P-V 1V_1#1" dans le nom de fichier
    match = re.search(pattern, file_name)
    if match:
        return match.group()
    else:
        return None

In [6]:
## Variables
output_folder = 'C:\\Users\\natha\\Downloads\\Semester_project\\Plots'
data_folder = 'C:\\Users\\natha\\Downloads\\Semester_project\\Data'
code_folder = 'C:\\Users\\natha\\Downloads\\Semester_project\\Code'
parameter_path = 'C:\\Users\\natha\\Downloads\\Semester_project\\Code\\Parameter.xlsx'

list_exp_1 = ["P-V 1V_1#1", "P-V 1V_2#1", "P-V 2V_1#1", "P-V 2V_2#1", "P-V 3V_1#1", "P-V 3V_2#1", 
                            "P-V 4V_1#1", "P-V 4V_2#1", "P-V 3V neg_1#1", "P-V 3V neg_1#1", "PUND 5V_1#1", "PUND 5V neg_1#1",
                            "CV 3V_1#1"]
size_list = ["50", "100", "150", "200", "500"]

In [7]:
## PARAMETER FILE : FIRST COL: SAMPLE ID, THEN VARIABLES, THEN SPACE, THEN SIZES

# Load parameter
parameter = pd.read_excel(parameter_path)
sample_IDs = parameter['sample ID'].to_list()
parameter.set_index('sample ID', inplace=True) # Set the first column (sample ID) as the index
print(parameter)

# Get the column names
parameter_names = parameter.columns
print('\nNumber of parameters:', len(parameter_names))
print('Parameter names:', ', '.join(map(str,parameter_names)))

          Sputtering method  Annealing temperature  Annealing time
sample ID                                                         
3dec01                   DP                    600              20
3dec13                   DP                    600             120
3dec05                   DP                    600             240
3dec17                   DP                    500              20
3dec11                   DP                    500             120
3dec03                   DP                    500             240
3dec15                   DP                    450              20
3dec09                   DP                    450             120
3dec07                   DP                    450             240
3dec18                   SP                    600              20
3dec12                   SP                    600             120
3dec04                   SP                    600             240
3dec16                   SP                    500            

In [8]:
def get_chips_from_experience(experience_string, param_df):
    experience_parts = experience_string.split('_')
    
    # Get the number of parameter
    nb_params = len(param_df.columns)
    nb_exp_parts = len(experience_parts)

    # Initialisation
    chip_list = []  

    # check that nb params = nb experience parts
    if nb_params == nb_exp_parts:
        df_temp = param_df
        for i in range(nb_params):
            chip_list = []
            param_col = df_temp.iloc[:,i]
            for j in range(len(param_col)):
                if (str(param_col.iloc[j]) == experience_parts[i]):
                    chip_list.append(param_col.index[j])
            #print(chip_list)
            df_temp = df_temp.loc[chip_list] # continue only with rows which have the correct parameter


    else:
        print('Error: Number of parameters are not equal')

    if chip_list is None:
        print('No chips for the experience', experience_string, 'found')

    return chip_list

def get_experience_from_chip(chip_name, param_df):
    experience_list = param_df.loc[chip_name].to_list()
    experience_string = '_'.join(map(str,experience_list))
    return experience_string

In [9]:
experience_df = parameter.loc[sample_IDs[1]]
print('Experience dataframe:\n ------------------')
print(experience_df)

experience = get_experience_from_chip(sample_IDs[1], parameter)
print('\n\nExperience String:\n ------------------\n',experience)

chips = get_chips_from_experience(experience, parameter)
print('\n\nChips corresponding to experience', experience, ':\n ------------------\n',chips)

Experience dataframe:
 ------------------
Sputtering method         DP
Annealing temperature    600
Annealing time           120
Name: 3dec13, dtype: object


Experience String:
 ------------------
 DP_600_120


Chips corresponding to experience DP_600_120 :
 ------------------
 ['3dec13']


In [21]:
def get_list_of_experiences(param_df):
    experience_list = []
    for i in range(len(param_df)):
        experience_string = '_'.join(map(str,param_df.iloc[i]))
        experience_list.append(experience_string)

    return experience_list

In [25]:
experience_list = get_list_of_experiences(parameter)
print('Number of experiences loaded:', len(experience_list))
print('------------------------------------')
print('Full list:\n', experience_list)

Number of experiences loaded: 18
------------------------------------
Full list:
 ['DP_600_20', 'DP_600_120', 'DP_600_240', 'DP_500_20', 'DP_500_120', 'DP_500_240', 'DP_450_20', 'DP_450_120', 'DP_450_240', 'SP_600_20', 'SP_600_120', 'SP_600_240', 'SP_500_20', 'SP_500_120', 'SP_500_120', 'SP_500_240', 'SP_450_20', 'SP_450_240']
